### Read caffe weights and save them in numpy

In [25]:
# code to convert MSGNET from caffe to npy
import caffe
import argparse
import numpy as np
from collections import OrderedDict

In [26]:
# parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument('--proto-path', type=str, default="./models/NYUV2/deploy.prototxt")
# parser.add_argument('--caffemodel-path', type=str, default="./models/NYUV2/cvpr_nyuv2.caffemodel")
# parser.add_argument('--output-path', type=str, default="./caffe_params.npy")
proto_path = "./models/NYUV2/deploy.prototxt"
caffemodel_path = "./models/NYUV2/cvpr_nyuv2.caffemodel"
npy_path = "./caffe_params_nyuv2.npy"

In [30]:
# args = parser.parse_args()
net = caffe.Net(proto_path, caffemodel_path, caffe.TRAIN)

layers = OrderedDict()
for name, params in net.params.items():
    layers[name] = {}
    # probably only works for DORN's caffe model...
    if "bn" in name: # batchnorm
        print("{} left: {}".format(name, len(params) - 4))
        layers[name]["scale"] = params[0].data
        layers[name]["shift"] = params[1].data
        layers[name]["mean"] = params[2].data
        layers[name]["var"] = params[3].data
    elif "ip" in name: # inner product (i.e. fully connected)
        print("{} left: {}".format(name, len(params) - 2))
        layers[name]["weight"] = params[0].data
        layers[name]["bias"] = params[1].data
    else: # convolution
        layers[name]["weight"] = params[0].data
        if len(params) == 2:
            print("{} left: {}".format(name, len(params) - 2))
            layers[name]["bias"] = params[1].data
        else:
            print("{} left: {}".format(name, len(params) - 1))
# Save layers to file
with open(npy_path, "w") as f:
    print("saving to {}".format(npy_path))
    np.save(f, layers)

conv1_1_3x3_s2 left: 0
conv1_1_3x3_s2/bn left: 0
conv1_2_3x3 left: 0
conv1_2_3x3/bn left: 0
conv1_3_3x3 left: 0
conv1_3_3x3/bn left: 0
conv2_1_1x1_reduce left: 0
conv2_1_1x1_reduce/bn left: 0
conv2_1_3x3 left: 0
conv2_1_3x3/bn left: 0
conv2_1_1x1_increase left: 0
conv2_1_1x1_increase/bn left: 0
conv2_1_1x1_proj left: 0
conv2_1_1x1_proj/bn left: 0
conv2_2_1x1_reduce left: 0
conv2_2_1x1_reduce/bn left: 0
conv2_2_3x3 left: 0
conv2_2_3x3/bn left: 0
conv2_2_1x1_increase left: 0
conv2_2_1x1_increase/bn left: 0
conv2_3_1x1_reduce left: 0
conv2_3_1x1_reduce/bn left: 0
conv2_3_3x3 left: 0
conv2_3_3x3/bn left: 0
conv2_3_1x1_increase left: 0
conv2_3_1x1_increase/bn left: 0
conv3_1_1x1_reduce left: 0
conv3_1_1x1_reduce/bn left: 0
conv3_1_3x3 left: 0
conv3_1_3x3/bn left: 0
conv3_1_1x1_increase left: 0
conv3_1_1x1_increase/bn left: 0
conv3_1_1x1_proj left: 0
conv3_1_1x1_proj/bn left: 0
conv3_2_1x1_reduce left: 0
conv3_2_1x1_reduce/bn left: 0
conv3_2_3x3 left: 0
conv3_2_3x3/bn left: 0
conv3_2_1x1_inc

PicklingError: Can't pickle <type 'numpy.ndarray'>: it's not the same object as numpy.ndarray

### Load numpy file and pytorch network

In [14]:
import numpy as np
import torch
import torch.nn as nn

In [31]:
layers = np.load(npy_path)
layers = layers[()] # Weird indexing thing
# print(layers['ip1/depth']["weight"].shape)
# print(layers["conv1_1_3x3_s2/bn"].keys())
new_layers = {}
for key in layers:
    new_key = key.replace("/", "_")
    new_layers[new_key] = layers[key]
layers = new_layers

EOFError: 

In [24]:
state_dict = {}
for layer_name in layers:
    if "bn" in layer_name:
        state_dict[layer_name + ".weight"] = torch.from_numpy(layers[layer_name]["scale"])
        state_dict[layer_name + ".bias"] = torch.from_numpy(layers[layer_name]["shift"])
        state_dict[layer_name + ".running_mean"] = torch.from_numpy(layers[layer_name]["mean"])
        state_dict[layer_name + ".running_var"] = torch.from_numpy(layers[layer_name]["var"])
    elif "ip" in layer_name:
        state_dict[layer_name + ".weight"] = torch.from_numpy(layers[layer_name]["weight"])
        state_dict[layer_name + ".weight"] = torch.from_numpy(layers[layer_name][])
    else:
        state_dict[layer_name + ".weight"]
        if len(layers[layer_name] == 2):


conv2_3_1x1_reduce
conv4_12_3x3
conv3_1_1x1_increase
conv5_2_3x3
conv1_2_3x3
conv4_20_3x3
conv3_2_1x1_reduce
conv4_23_1x1_increase
conv4_7_3x3
conv4_3_3x3
conv4_5_3x3
conv5_1_1x1_reduce
conv4_14_1x1_reduce
conv4_22_1x1_increase
conv4_19_1x1_reduce
conv4_4_1x1_increase
conv4_17_3x3
conv4_8_1x1_increase
conv2_2_3x3
conv4_16_1x1_increase
conv2_2_1x1_increase
aspp_1_soft
conv2_2_1x1_reduce
conv6_4_soft
conv6_5_soft
conv4_11_3x3
conv3_1_1x1_proj
conv4_9_1x1_reduce
conv4_1_1x1_proj
conv4_21_1x1_reduce
conv4_20_1x1_increase
conv4_19_3x3
conv4_15_1x1_reduce
conv4_21_3x3
conv4_10_3x3
conv3_1_3x3
conv4_7_1x1_increase
conv5_1_1x1_proj
conv4_1_1x1_reduce
conv1_3_3x3
conv3_4_1x1_reduce
aspp_3_soft
conv4_6_3x3
conv4_22_1x1_reduce
conv4_18_1x1_reduce
conv3_4_1x1_increase
conv5_2_1x1_increase
conv6_1_soft
conv2_1_1x1_reduce
conv4_23_3x3
conv4_3_1x1_reduce
conv5_3_1x1_reduce
conv4_2_3x3
conv2_3_1x1_increase
conv4_4_3x3
conv4_6_1x1_reduce
conv4_11_1x1_reduce
conv3_2_1x1_increase
conv1_1_3x3_s2
conv4_9_1

In [4]:
from DORN_pytorch import DORN_nyu

In [6]:
net = DORN_nyu()

In [8]:
net.load_state_dict(layers)

RuntimeError: Error(s) in loading state_dict for DORN_nyu:
	Missing key(s) in state_dict: "conv1_1_3x3_s2.weight", "conv1_1_3x3_s2_bn.running_var", "conv1_1_3x3_s2_bn.bias", "conv1_1_3x3_s2_bn.weight", "conv1_1_3x3_s2_bn.running_mean", "conv1_2_3x3.weight", "conv1_2_3x3_bn.running_var", "conv1_2_3x3_bn.bias", "conv1_2_3x3_bn.weight", "conv1_2_3x3_bn.running_mean", "conv1_3_3x3.weight", "conv1_3_3x3_bn.running_var", "conv1_3_3x3_bn.bias", "conv1_3_3x3_bn.weight", "conv1_3_3x3_bn.running_mean", "conv2_1_1x1_reduce.weight", "conv2_1_1x1_reduce_bn.running_var", "conv2_1_1x1_reduce_bn.bias", "conv2_1_1x1_reduce_bn.weight", "conv2_1_1x1_reduce_bn.running_mean", "conv2_1_3x3.weight", "conv2_1_3x3_bn.running_var", "conv2_1_3x3_bn.bias", "conv2_1_3x3_bn.weight", "conv2_1_3x3_bn.running_mean", "conv2_1_1x1_increase.weight", "conv2_1_1x1_increase_bn.running_var", "conv2_1_1x1_increase_bn.bias", "conv2_1_1x1_increase_bn.weight", "conv2_1_1x1_increase_bn.running_mean", "conv2_1_1x1_proj.weight", "c